In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from random import shuffle
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten 
from keras.layers import Dense, Dropout
import keras.utils as image
from keras.preprocessing.image import ImageDataGenerator
#import split_folders

In [2]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.summary()
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 32)       0

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [6]:
TRAINING_DIR = 'final_data/train'
VALIDATION_DIR = 'final_data/val'

In [7]:
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=16,
    class_mode='binary',
    target_size=(224, 224))

Found 1100 images belonging to 2 classes.


In [8]:
validation_datagen = ImageDataGenerator(
                        rescale=1. / 255)

In [9]:
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size=16,
    class_mode='binary',
    target_size=(224, 224))


Found 276 images belonging to 2 classes.


In [10]:
classifier.fit(train_generator,
            epochs=15,
            steps_per_epoch=1100/16,
            verbose=1,
            validation_data=validation_generator,
            validation_steps=276/16)

Epoch 1/15
68/68 [==============================] - 41s 585ms/step - loss: 0.6571 - accuracy: 0.6745 - val_loss: 0.2944 - val_accuracy: 0.9312
Epoch 2/15
68/68 [==============================] - 41s 594ms/step - loss: 0.3362 - accuracy: 0.8764 - val_loss: 0.1563 - val_accuracy: 0.9384
Epoch 3/15
68/68 [==============================] - 41s 601ms/step - loss: 0.2735 - accuracy: 0.9064 - val_loss: 0.1279 - val_accuracy: 0.9420
Epoch 4/15
68/68 [==============================] - 41s 592ms/step - loss: 0.2344 - accuracy: 0.9127 - val_loss: 0.1208 - val_accuracy: 0.9493
Epoch 5/15
68/68 [==============================] - 40s 584ms/step - loss: 0.2131 - accuracy: 0.9264 - val_loss: 0.1031 - val_accuracy: 0.9565
Epoch 6/15
68/68 [==============================] - 41s 602ms/step - loss: 0.1653 - accuracy: 0.9473 - val_loss: 0.0614 - val_accuracy: 0.9783
Epoch 7/15
68/68 [==============================] - 42s 609ms/step - loss: 0.1891 - accuracy: 0.9318 - val_loss: 0.0934 - val_accuracy: 0.9601

In [11]:
classifier.save("mask_detector_new_generated.h5", save_format="h5")

In [13]:
for test_image in os.listdir('testing/'):
    test_image = image.load_img('testing/'+test_image, target_size = (224, 224))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    print(result[0][0])
    if result[0][0] == 1:
        prediction = "Not Masked!"
        print(prediction)
    else: 
        prediction = "Masked"
        print(prediction)
    

1/1 [==============================] - 0s 28ms/step
0.0
Masked
1/1 [==============================] - 0s 26ms/step
3.0542838e-18
Masked
1/1 [==============================] - 0s 28ms/step
0.0
Masked
1/1 [==============================] - 0s 27ms/step
1.0
Not Masked!
1/1 [==============================] - 0s 25ms/step
1.0
Not Masked!
1/1 [==============================] - 0s 27ms/step
1.0
Not Masked!


In [15]:
##optional for live recognization

import cv2
from keras.models import load_model
import numpy as np

model = load_model("mask_detector_new_generated.h5")

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

webcam = cv2.VideoCapture(0)

while True:
    ret, frame = webcam.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        face = frame[y:y+h, x:x+w]

        face = cv2.resize(face, (224, 224))

        face = np.expand_dims(face, axis=0)

        face = face / 255.0

        result = model.predict(face)

        prediction = "Masked" if result[0][0] < 0.5 else "Not Masked"

        cv2.putText(frame, prediction, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Webcam", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()

cv2.destroyAllWindows()


1/1 [==============================] - 0s 27ms/step
